In [1]:
import getpass
import os

os.environ["DASHSCOPE_API_KEY"] = 'sk-a31a563bce7146bf8c8ab93ad5fea537'
os.environ["TAVILY_API_KEY"] = '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'

In [2]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_result=5)

python_repl_tool = PythonREPLTool()

In [71]:
from langchain_core.messages import HumanMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)
    print('--------------------')
    print(result)
    print('********************')
    return {
        "messages": [HumanMessage(content=result['messages'][-1].content, name=name)]
        # "messages": result['messages']
    }

In [72]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

members = ['Researcher', 'Coder']
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
    " Given the conversation below, who should act next?"
    " Or should we FINISH? Select one of: {options}"
)
options = ['FINISH'] + members

class routeResponse(BaseModel):
    next: Literal[*options]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name='messages')
    ]
).partial(options=str(options), members=','.join(members))

# 使用 Qwen 模型
qwen_llm = ChatTongyi(
    model="qwen-max",              # Qwen 模型名称，例如 "qwen-7b" 或 "qwen-14b"
    api_key="sk-a31a563bce7146bf8c8ab93ad5fea537"   # 替换为实际的 API 密钥
)

def supervisor_agent(state):
    supervisor_chain = prompt | qwen_llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

In [73]:
import functools
import operator
from typing import Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

research_agent = create_react_agent(qwen_llm, tools=[tavily_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_react_agent(qwen_llm, tools=[python_repl_tool])
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_agent)

In [74]:
for member in members:
    workflow.add_edge(member, 'supervisor')
conditional_map = {k: k for k in members}
conditional_map['FINISH'] = END
workflow.add_conditional_edges('supervisor', lambda x: x['next'], conditional_map)
workflow.add_edge(START, 'supervisor')

graph = workflow.compile()

In [76]:
# for s in graph.stream(
#     {
#         "messages": [
#             HumanMessage(content="Code hello world and print it to the terminal")
#         ]
#     }
# ):
#     if "__end__" not in s:
#         print(s)
#         print("----")

res = graph.invoke(
    {
        "messages": [
            HumanMessage(content="Code hello world and print it to the terminal", role='user')
            # "Code hello world and print it to the terminal"
        ]
    }
)
print(res)

--------------------
{'messages': [HumanMessage(content='Code hello world and print it to the terminal', additional_kwargs={}, response_metadata={}, id='62f51810-2cb4-45e0-9b79-4223c0e0ba3f', role='user'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'Python_REPL', 'arguments': '{"query": "print(\'Hello, world!\')"}'}, 'index': 0, 'id': 'call_1f55addafa0c43989935c9', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '5800608b-077f-928d-9c18-be3ed6f26aff', 'token_usage': {'input_tokens': 205, 'output_tokens': 24, 'total_tokens': 229}}, id='run-ba0f2b7a-3554-4b92-8bfa-e9cd6e6144d9-0', tool_calls=[{'name': 'Python_REPL', 'args': {'query': "print('Hello, world!')"}, 'id': 'call_1f55addafa0c43989935c9', 'type': 'tool_call'}]), ToolMessage(content='Hello, world!\n', name='Python_REPL', id='e3e84f93-e861-44a2-b7bb-20d27b391ebc', tool_call_id='call_1f55addafa0c43989935c9'), AIMessage(content='The me